### Beginning

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
from former import Tab_Former, log_rmse, run_model, Encoder

In [3]:
clean = pd.read_csv("Flight_prediction\Clean_Dataset.csv")
economy = pd.read_csv("Flight_prediction\economy.csv")
businesss = pd.read_csv("Flight_prediction/business.csv")

In [4]:
clean.head()

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [5]:
economy.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


In [6]:
economy.shape

(206774, 11)

In [7]:
businesss.tail()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
93482,31-03-2022,Vistara,UK,822,09:45,Chennai,10h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:50,Hyderabad,"69,265"
93483,31-03-2022,Vistara,UK,826,12:30,Chennai,10h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:55,Hyderabad,"77,105"
93484,31-03-2022,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"79,099"
93485,31-03-2022,Vistara,UK,828,07:00,Chennai,10h 00m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,17:00,Hyderabad,"81,585"
93486,31-03-2022,Vistara,UK,822,09:45,Chennai,10h 05m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:50,Hyderabad,"81,585"


In [8]:
businesss.shape

(93487, 11)

In [9]:
len(clean['flight'].unique())

1561

### Preprocessing

In [10]:
df = pd.concat([economy,businesss],axis=0)

In [11]:
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


In [12]:
df['date'] = pd.to_datetime(df['date'],format = "%d-%m-%Y")

In [13]:
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,2022-02-11,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,2022-02-11,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,2022-02-11,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,2022-02-11,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,2022-02-11,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


In [14]:
# df.set_index('date',inplace=True)
# df.head()

In [15]:
def stop_data(data):
    for i in data:
        if 'non-stop' in i:
            return 'zero'
        elif '1' in i:
            return 'one'
        elif '2' in i:
            return 'two'

    return data

In [16]:
df['stop'] = df['stop'].apply(stop_data)

In [17]:
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,2022-02-11,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,2022-02-11,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,2022-02-11,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,2022-02-11,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,2022-02-11,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


In [18]:
df['arr_time'] = pd.to_datetime(df['arr_time'],format = "%H:%M")

In [19]:
df['dep_time'] = pd.to_datetime(df['dep_time'],format = "%H:%M")
# df['dep_time'].dt.time

In [20]:
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,2022-02-11,SpiceJet,SG,8709,1900-01-01 18:55:00,Delhi,02h 10m,non-stop,1900-01-01 21:05:00,Mumbai,"5,953"
1,2022-02-11,SpiceJet,SG,8157,1900-01-01 06:20:00,Delhi,02h 20m,non-stop,1900-01-01 08:40:00,Mumbai,"5,953"
2,2022-02-11,AirAsia,I5,764,1900-01-01 04:25:00,Delhi,02h 10m,non-stop,1900-01-01 06:35:00,Mumbai,"5,956"
3,2022-02-11,Vistara,UK,995,1900-01-01 10:20:00,Delhi,02h 15m,non-stop,1900-01-01 12:35:00,Mumbai,"5,955"
4,2022-02-11,Vistara,UK,963,1900-01-01 08:50:00,Delhi,02h 20m,non-stop,1900-01-01 11:10:00,Mumbai,"5,955"


In [21]:
# pd.cut(x = df['dep_time'], bins = bins, labels=labels)

In [22]:
df

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,2022-02-11,SpiceJet,SG,8709,1900-01-01 18:55:00,Delhi,02h 10m,non-stop,1900-01-01 21:05:00,Mumbai,"5,953"
1,2022-02-11,SpiceJet,SG,8157,1900-01-01 06:20:00,Delhi,02h 20m,non-stop,1900-01-01 08:40:00,Mumbai,"5,953"
2,2022-02-11,AirAsia,I5,764,1900-01-01 04:25:00,Delhi,02h 10m,non-stop,1900-01-01 06:35:00,Mumbai,"5,956"
3,2022-02-11,Vistara,UK,995,1900-01-01 10:20:00,Delhi,02h 15m,non-stop,1900-01-01 12:35:00,Mumbai,"5,955"
4,2022-02-11,Vistara,UK,963,1900-01-01 08:50:00,Delhi,02h 20m,non-stop,1900-01-01 11:10:00,Mumbai,"5,955"
...,...,...,...,...,...,...,...,...,...,...,...
93482,2022-03-31,Vistara,UK,822,1900-01-01 09:45:00,Chennai,10h 05m,one,1900-01-01 19:50:00,Hyderabad,"69,265"
93483,2022-03-31,Vistara,UK,826,1900-01-01 12:30:00,Chennai,10h 25m,one,1900-01-01 22:55:00,Hyderabad,"77,105"
93484,2022-03-31,Vistara,UK,832,1900-01-01 07:05:00,Chennai,13h 50m,one,1900-01-01 20:55:00,Hyderabad,"79,099"
93485,2022-03-31,Vistara,UK,828,1900-01-01 07:00:00,Chennai,10h 00m,one,1900-01-01 17:00:00,Hyderabad,"81,585"


In [23]:
timestamp = pd.Timedelta('02:00:00')
time = pd.to_datetime('20:00:00',format='%H:%M:%S')


if time >= pd.to_datetime('00:00:00',format='%H:%M:%S') and time < pd.to_datetime('05:00:00',format='%H:%M:%S'):
    print( "Early Morning")
elif time >= pd.to_datetime('05:00:00',format='%H:%M:%S') and time < pd.to_datetime('12:00:00',format='%H:%M:%S'):
    print( "Morning")
elif time >= pd.to_datetime('12:00:00',format='%H:%M:%S') and time < pd.to_datetime('19:00:00',format='%H:%M:%S'):
    print( "Afternoon")
elif time >= pd.to_datetime('19:00:00',format='%H:%M:%S') and time < pd.to_datetime('21:00:00',format='%H:%M:%S'):
    print( "Night")
    # print(time.time)
else:
    print( "Late Night")

Night


In [24]:
def categorize_time(time_series):
    categories = []
    for time in time_series.dt.time:
        time = pd.to_datetime(time,format="%H:%M:%S")
        if time >= pd.to_datetime('00:00:00',format="%H:%M:%S") and time < pd.to_datetime('05:00:00',format="%H:%M:%S"):
            categories.append("Early Morning")
        elif time >= pd.to_datetime('05:00:00',format="%H:%M:%S") and time < pd.to_datetime('12:00:00',format="%H:%M:%S"):
            categories.append("Morning")        
        elif time >= pd.to_datetime('12:00:00',format="%H:%M:%S") and time < pd.to_datetime('19:00:00',format="%H:%M:%S"):
            categories.append("Afternoon")        
        elif time >= pd.to_datetime('19:00:00',format="%H:%M:%S") and time < pd.to_datetime('21:00:00',format="%H:%M:%S"):
            categories.append("Night")        
        else:
            categories.append("Late Night")
        

    return categories
        


In [25]:
dep = categorize_time(df['dep_time'])

In [26]:
arr = categorize_time(df['arr_time'])

In [27]:
df['Departure_time'] = pd.DataFrame(dep)
# df['Departure_time'] = de.values
df['Arrival_time'] = pd.DataFrame(arr)

In [28]:
df.tail()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,Departure_time,Arrival_time
93482,2022-03-31,Vistara,UK,822,1900-01-01 09:45:00,Chennai,10h 05m,one,1900-01-01 19:50:00,Hyderabad,"69,265",Night,Early Morning
93483,2022-03-31,Vistara,UK,826,1900-01-01 12:30:00,Chennai,10h 25m,one,1900-01-01 22:55:00,Hyderabad,"77,105",Morning,Afternoon
93484,2022-03-31,Vistara,UK,832,1900-01-01 07:05:00,Chennai,13h 50m,one,1900-01-01 20:55:00,Hyderabad,"79,099",Afternoon,Early Morning
93485,2022-03-31,Vistara,UK,828,1900-01-01 07:00:00,Chennai,10h 00m,one,1900-01-01 17:00:00,Hyderabad,"81,585",Night,Early Morning
93486,2022-03-31,Vistara,UK,822,1900-01-01 09:45:00,Chennai,10h 05m,one,1900-01-01 19:50:00,Hyderabad,"81,585",Morning,Afternoon


In [29]:
df.dtypes

date              datetime64[ns]
airline                   object
ch_code                   object
num_code                   int64
dep_time          datetime64[ns]
from                      object
time_taken                object
stop                      object
arr_time          datetime64[ns]
to                        object
price                     object
Departure_time            object
Arrival_time              object
dtype: object

In [30]:
def convert_to_hours(duration_str):
    """
    Convert a duration string in the format "Xh Ym" or "Xh m" or "Xh" to total hours.

    Args:
        duration_str (str): Duration string in the format "Xh Ym" or "Xh m" or "Xh".

    Returns:
        float: Total duration in hours.
    """
    hours, minutes = duration_str.split('h')
    # print(hours)
    hours = float(hours)
    if minutes.strip():
        # print(minutes)
        minutes = (minutes.replace('m', ''))
        if minutes == ' ':
            minutes = 0
    else:
        minutes = 0
    total_hours = hours + int(minutes) / 60.0
    return round(total_hours,2)
    

In [31]:
convert_to_hours('02h 10m')

2.17

In [32]:
df['duration_time'] = df['time_taken'].apply(convert_to_hours)
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,Departure_time,Arrival_time,duration_time
0,2022-02-11,SpiceJet,SG,8709,1900-01-01 18:55:00,Delhi,02h 10m,non-stop,1900-01-01 21:05:00,Mumbai,"5,953",Afternoon,Late Night,2.17
1,2022-02-11,SpiceJet,SG,8157,1900-01-01 06:20:00,Delhi,02h 20m,non-stop,1900-01-01 08:40:00,Mumbai,"5,953",Morning,Morning,2.33
2,2022-02-11,AirAsia,I5,764,1900-01-01 04:25:00,Delhi,02h 10m,non-stop,1900-01-01 06:35:00,Mumbai,"5,956",Early Morning,Morning,2.17
3,2022-02-11,Vistara,UK,995,1900-01-01 10:20:00,Delhi,02h 15m,non-stop,1900-01-01 12:35:00,Mumbai,"5,955",Morning,Afternoon,2.25
4,2022-02-11,Vistara,UK,963,1900-01-01 08:50:00,Delhi,02h 20m,non-stop,1900-01-01 11:10:00,Mumbai,"5,955",Morning,Morning,2.33


In [33]:
max_duration = df['duration_time'].max()

In [34]:
df[df['duration_time'] == df['duration_time'].max()]

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,Departure_time,Arrival_time,duration_time
193997,2022-02-12,Air India,AI,672,1900-01-01 16:05:00,Chennai,49h 50m,two,1900-01-01 17:55:00,Bangalore,"23,891",Afternoon,Afternoon,49.83
194467,2022-02-19,Air India,AI,672,1900-01-01 16:05:00,Chennai,49h 50m,one,1900-01-01 17:55:00,Bangalore,"17,538",Afternoon,Afternoon,49.83


In [35]:
economy_count = economy.shape[0]

# Create the 'class' column with default value 'business'
df['class'] = 'business'

# Update the 'class' column for the first 'economy_count' rows to 'economy'
df.loc[:economy_count-1, 'class'] = 'economy'

In [36]:
df['class'].unique()

array(['economy', 'business'], dtype=object)

In [37]:
df.tail()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,Departure_time,Arrival_time,duration_time,class
93482,2022-03-31,Vistara,UK,822,1900-01-01 09:45:00,Chennai,10h 05m,one,1900-01-01 19:50:00,Hyderabad,"69,265",Night,Early Morning,10.08,business
93483,2022-03-31,Vistara,UK,826,1900-01-01 12:30:00,Chennai,10h 25m,one,1900-01-01 22:55:00,Hyderabad,"77,105",Morning,Afternoon,10.42,business
93484,2022-03-31,Vistara,UK,832,1900-01-01 07:05:00,Chennai,13h 50m,one,1900-01-01 20:55:00,Hyderabad,"79,099",Afternoon,Early Morning,13.83,business
93485,2022-03-31,Vistara,UK,828,1900-01-01 07:00:00,Chennai,10h 00m,one,1900-01-01 17:00:00,Hyderabad,"81,585",Night,Early Morning,10.00,business
93486,2022-03-31,Vistara,UK,822,1900-01-01 09:45:00,Chennai,10h 05m,one,1900-01-01 19:50:00,Hyderabad,"81,585",Morning,Afternoon,10.08,business


In [38]:
df.drop(['date','num_code','dep_time','time_taken','arr_time'],axis=1,inplace=True)

In [39]:
df.head()

,airline,ch_code,from,stop,to,price,Departure_time,Arrival_time,duration_time,class
0,SpiceJet,SG,Delhi,non-stop,Mumbai,"5,953",Afternoon,Late Night,2.17,economy
1,SpiceJet,SG,Delhi,non-stop,Mumbai,"5,953",Morning,Morning,2.33,economy
2,AirAsia,I5,Delhi,non-stop,Mumbai,"5,956",Early Morning,Morning,2.17,economy
3,Vistara,UK,Delhi,non-stop,Mumbai,"5,955",Morning,Afternoon,2.25,economy
4,Vistara,UK,Delhi,non-stop,Mumbai,"5,955",Morning,Morning,2.33,economy


In [40]:
df['class'].value_counts()

class
economy     206774
business     93487
Name: count, dtype: int64

In [41]:
from imblearn.over_sampling import SMOTE

In [42]:
smote = SMOTE(random_state=42)

### PCA plot


In [ ]:
from sklearn.decomposition import PCA

def plot_words(embedding, sequences, sentence):
    pca = PCA(n_components=2)
    X_pca_train = pca.fit_transform(embedding[sentence,0:len(sequences[sentence]),:])


    fig, ax = plt.subplots(figsize=(12, 6)) 
    plt.rcParams['font.size'] = '12'
    ax.scatter(X_pca_train[:, 0], X_pca_train[:, 1])
    words = list(word_index.keys())
    for i, index in enumerate(sequences[sentence]):
        ax.annotate(words[index-1], (X_pca_train[i, 0], X_pca_train[i, 1]))

plot_words(embedding, sequences, 0)

In [ ]:
plot_words(embedding, sequences, 0)

In [ ]:
df.shape

(300261, 10)

In [ ]:
df.describe()

,duration_time
count,300261.000000
mean,12.217794
std,7.192961
min,0.830000
25%,6.750000
50%,11.250000
75%,16.170000
max,49.830000


In [44]:
df['ch_code'].unique()

array(['SG', 'I5', 'UK', 'G8', '6E', 'AI', '2T', 'S5'], dtype=object)

In [45]:
df.head()

,airline,ch_code,from,stop,to,price,Departure_time,Arrival_time,duration_time,class
0,SpiceJet,SG,Delhi,non-stop,Mumbai,"5,953",Afternoon,Late Night,2.17,economy
1,SpiceJet,SG,Delhi,non-stop,Mumbai,"5,953",Morning,Morning,2.33,economy
2,AirAsia,I5,Delhi,non-stop,Mumbai,"5,956",Early Morning,Morning,2.17,economy
3,Vistara,UK,Delhi,non-stop,Mumbai,"5,955",Morning,Afternoon,2.25,economy
4,Vistara,UK,Delhi,non-stop,Mumbai,"5,955",Morning,Morning,2.33,economy


In [46]:
clean['class'].value_counts()

class
Economy     206666
Business     93487
Name: count, dtype: int64

In [47]:
clean.tail()

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
300148,300148,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,69265
300149,300149,Vistara,UK-826,Chennai,Afternoon,one,Night,Hyderabad,Business,10.42,49,77105
300150,300150,Vistara,UK-832,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,300151,Vistara,UK-828,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585
300152,300152,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,81585


In [48]:
clean.describe(include = 'all')

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
count,300153.000000,300153,300153,300153,300153,300153,300153,300153,300153,300153.000000,300153.000000,300153.000000
unique,NaN,6,1561,6,6,3,6,6,2,NaN,NaN,NaN
top,NaN,Vistara,UK-706,Delhi,Morning,one,Night,Mumbai,Economy,NaN,NaN,NaN
freq,NaN,127859,3235,61343,71146,250863,91538,59097,206666,NaN,NaN,NaN
mean,150076.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.221021,26.004751,20889.660523
std,86646.852011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.191997,13.561004,22697.767366
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.830000,1.000000,1105.000000
25%,75038.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.830000,15.000000,4783.000000
50%,150076.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.250000,26.000000,7425.000000
75%,225114.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.170000,38.000000,42521.000000


In [49]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        300153 non-null  int64  
 1   airline           300153 non-null  object 
 2   flight            300153 non-null  object 
 3   source_city       300153 non-null  object 
 4   departure_time    300153 non-null  object 
 5   stops             300153 non-null  object 
 6   arrival_time      300153 non-null  object 
 7   destination_city  300153 non-null  object 
 8   class             300153 non-null  object 
 9   duration          300153 non-null  float64
 10  days_left         300153 non-null  int64  
 11  price             300153 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 27.5+ MB


In [50]:
df.head()

,airline,ch_code,from,stop,to,price,Departure_time,Arrival_time,duration_time,class
0,SpiceJet,SG,Delhi,non-stop,Mumbai,"5,953",Afternoon,Late Night,2.17,economy
1,SpiceJet,SG,Delhi,non-stop,Mumbai,"5,953",Morning,Morning,2.33,economy
2,AirAsia,I5,Delhi,non-stop,Mumbai,"5,956",Early Morning,Morning,2.17,economy
3,Vistara,UK,Delhi,non-stop,Mumbai,"5,955",Morning,Afternoon,2.25,economy
4,Vistara,UK,Delhi,non-stop,Mumbai,"5,955",Morning,Morning,2.33,economy


In [51]:
df.reset_index(inplace=True)

In [52]:
tab = Tab_Former(combined_df = df, target = "price")

In [53]:
from former import preprocess_data

In [54]:
cat_col, num_col = tab.cat_num_split(df.drop('price',axis=1))

In [55]:
print(cat_col)

['airline', 'ch_code', 'from', 'stop', 'to', 'Departure_time', 'Arrival_time', 'class']


In [56]:
feature_map = tab.feature_map()
mapp = len(feature_map)

In [57]:
feature_map

{'SpiceJet': 0,
 'AirAsia': 1,
 'Vistara': 2,
 'GO FIRST': 3,
 'Indigo': 4,
 'Air India': 5,
 'Trujet': 6,
 'StarAir': 7,
 'SG': 8,
 'I5': 9,
 'UK': 10,
 'G8': 11,
 '6E': 12,
 'AI': 13,
 '2T': 14,
 'S5': 15,
 'Delhi': 30,
 'Mumbai': 25,
 'Bangalore': 26,
 'Kolkata': 27,
 'Hyderabad': 28,
 'Chennai': 29,
 'non-stop ': 22,
 'one': 23,
 'two': 24,
 'Afternoon': 38,
 'Morning': 37,
 'Early Morning': 40,
 'Night': 39,
 'Late Night': 36,
 'economy': 41,
 'business': 42}

In [58]:
from sklearn.decomposition import PCA

def plot_words(embedding, sequences, sentence):
    pca = PCA(n_components=2)
    X_pca_train = pca.fit_transform(embedding[sentence,0:len(sequences[sentence]),:])


    fig, ax = plt.subplots(figsize=(12, 6)) 
    plt.rcParams['font.size'] = '12'
    ax.scatter(X_pca_train[:, 0], X_pca_train[:, 1])
    words = list(feature_map.keys())
    for i, index in enumerate(sequences[sentence]):
        ax.annotate(words[index-1], (X_pca_train[i, 0], X_pca_train[i, 1]))



In [59]:
print(feature_map)

{'SpiceJet': 0, 'AirAsia': 1, 'Vistara': 2, 'GO FIRST': 3, 'Indigo': 4, 'Air India': 5, 'Trujet': 6, 'StarAir': 7, 'SG': 8, 'I5': 9, 'UK': 10, 'G8': 11, '6E': 12, 'AI': 13, '2T': 14, 'S5': 15, 'Delhi': 30, 'Mumbai': 25, 'Bangalore': 26, 'Kolkata': 27, 'Hyderabad': 28, 'Chennai': 29, 'non-stop ': 22, 'one': 23, 'two': 24, 'Afternoon': 38, 'Morning': 37, 'Early Morning': 40, 'Night': 39, 'Late Night': 36, 'economy': 41, 'business': 42}


In [60]:
from sklearn.model_selection import GridSearchCV

In [61]:
from former import Scaling

In [62]:
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.linear_model import LinearRegression
rfr = RandomForestRegressor(n_estimators=200,max_leaf_nodes=60)
IS = IsolationForest()
lr = LinearRegression()

In [63]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import catboost as cb
from sklearn.neural_network import MLPRegressor

In [64]:
models = {
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'LightGBM': lgb.LGBMRegressor(),
    'CatBoost': cb.CatBoostRegressor(silent=True),
    "LinearRegression" : LinearRegression(),
    "MLP" : MLPRegressor()
}


In [65]:
def results():
    result = []
    for name, model in models.items():
        mod = run_model(model,train,target)
        df = pd.DataFrame([run_model(model,train,target)])
        result.append(mod)

    df = pd.DataFrame(result)
    df.set_index('Model Name',inplace=True)
    
    plt.figure(figsize = (12,8))
    df['rmse'].plot()
    plt.title("RMSE per each model")
    plt.ylabel("RMSE")
    plt.show()

    return df

### Expand blocks Layer in Encoder `1`, `2`, `4`, `8`, `12`, `50`, `100`.

#### Layer 1

In [66]:
d_model = len(cat_col)
print(d_model)

8


In [67]:
num_layer = 1

In [68]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=num_layer,d_ff=128,dropout=0.8)
enc_val = tab.encode()

In [ ]:
plot_words(enc_val, cat_col, 0)

In [ ]:
tab.tsne_plot(enc_val,700)

In [ ]:
tab.tsne_plot(enc_val,200)

In [ ]:
enc_val

In [ ]:
tab.tsne_plot(enc_val,-1)

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

In [ ]:
df1 = results()

In [ ]:
df1.add_prefix((f"{num_layer}"), axis=1)

#### Layer 2

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=2,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 4

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=4,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 8

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 12

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=12,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 50

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=50,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 100

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=100,d_ff=128,dropout=0.5)
enc_val = tab.encode()

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

### Scale the numerical values `min_max_scaling`, `z_score_scaling`, `robust_scaling`.

#### Encoding before scaling

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

#### min_max_scaling

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
target

: 

In [ ]:
train

: 

#### z_score_scaling 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Robust Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler

: 

In [ ]:
rs = RobustScaler()

: 

In [ ]:
scale = Scaling.robust_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

### Dropout in Encoder `0.2`, `0.4`, `0.8`.

#### Encoding with 0.2 dropout

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.2)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Encoding with 0.4 dropout

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.4)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Encoding with 0.8 dropout

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=8,d_ff=128,dropout=0.8)
enc_val = tab.encode(encoder = encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Dropout 0.5

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=27,num_layers=8,d_ff=128,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

### Another

#### min_max_scaling

In [ ]:
scale = Scaling.min_max_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### z_score_scaling 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Robust Scaling

In [ ]:
scale = Scaling.robust_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

### Expand Hidden Layer in Encoder `50`, `128`, `256`, `512`, `768`, `1024`, `2048`.

#### Layer 50

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=50,dropout=0.5)
enc_val = tab.encode(encoder = encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 128 

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=128,dropout=0.5)
enc_val = tab.encode(encoder = encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 256

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=256,dropout=0.5)
enc_val = tab.encode(encoder = encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 512

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=512,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 768

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=768,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

#### Layer 1024

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=1024,dropout=0.5)
enc_val = tab.encode(encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
target

: 

In [ ]:
train

: 

In [ ]:
results()

: 

#### Layer 2048

In [ ]:
%%time
torch.manual_seed(24)
encoder = Encoder(d_model=d_model,num_layers=5,d_ff=2048,dropout=0.5)
enc_val = tab.encode(encoder=encoder)

: 

In [ ]:
tab.tsne_plot(enc_val,700)

: 

In [ ]:
tab.tsne_plot(enc_val,200)

: 

In [ ]:
enc_val

: 

In [ ]:
tab.tsne_plot(enc_val,-1)

: 

In [ ]:
scale = Scaling.z_score_scaling
train, test = tab.preprocess(scaling = scale,encoded_values=enc_val)

: 

In [ ]:
results()

: 

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=len(cat_col), truncation=True, padding='max_length')
    outputs = model(**inputs)
    # Taking the mean of the embeddings from the last hidden state
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Encode categorical features
# cat_features = ['categorical_feature1', 'categorical_feature2']

: 

In [ ]:
encoded_feature = combined_df[cat_col]
encoded_feature.head()    

: 

In [ ]:
import pandas as pd

# Example DataFrame
data = {
    'categorical_feature1': ['cat1', 'cat2', 'cat1'],
    'categorical_feature2': ['cat3', 'cat4', 'cat3'],
    'numerical_feature1': [1.0, 2.0, 3.0],
    'numerical_feature2': [4.0, 5.0, 6.0],
    'target': [0, 1, 0]
}

df = pd.DataFrame(data)
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_embeddings(text):
    text = ' '.join([str(text) for feature in text])
    inputs = tokenizer(text, return_tensors='pt', max_length=3, truncation=True, padding='max_length')
    outputs = model(**inputs)
    # Taking the mean of the embeddings from the last hidden state
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


: 

In [ ]:
train_df

: 

In [ ]:
# cat_features = ['categorical_feature1', 'categorical_feature2']
encoded_features = []

for feature in cat_col:
    encoded_feature = get_bert_embeddings(train_df[feature])
    encoded_features.append(encoded_feature)

# Concatenate encoded categorical features
encoded_cat_df = pd.DataFrame(encoded_features[0].tolist()).add_prefix(f"{cat_col[0]}_emb_")
for i, feature in enumerate(cat_col[1:]):
    encoded_cat_df = pd.concat([encoded_cat_df, pd.DataFrame(encoded_features[i+1].tolist()).add_prefix(f"{feature}_emb_")], axis=1)
# Numerical features
# numerical_features = ['numerical_feature1', 'numerical_feature2']
numerical_df = train_df[num_col]

# Concatenate encoded categorical features with numerical features
final_df = pd.concat([encoded_cat_df, numerical_df], axis=1)

: 

In [ ]:
tab = Tab_Former(train_df=train_df,test_df=None, target = "SalePrice")
tab.preprocess_data()

: 

In [ ]:
final_df.head()

: 

In [ ]:
missing=final_df.isnull().sum().sort_values(ascending=False)
missing=missing.drop(missing[missing==0].index)
df = pd.DataFrame(missing)
label = train_df['SalePrice']
# test_id = test_df['Id']
# train_df.drop("Id",inplace=True,axis=1)
final_df.drop(missing.keys(),inplace=True,axis=1)

: 

In [ ]:
len(final_df.isna().sum())

: 

: 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Target variable
# y = df['target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(final_df, label, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestRegressor()
clf.fit(X_train, y_train)

# Evaluate the model
accuracy = clf.score(X_test, y_test)
print(f"Model Accuracy: {accuracy}")


: 

In [ ]:
for i in range(combined_df.shape[0]):
    get_bert_embeddings(combined_df[cat_col].iloc[i])

: 

In [ ]:
encoded_features = []

for feature in cat_col:
    encoded_feature = get_bert_embeddings(combined_df[feature].)
    encoded_features.append(encoded_feature)

# Concatenate encoded categorical features
encoded_cat_df = pd.DataFrame(encoded_features[0].tolist()).add_prefix(f"{cat_col[0]}_emb_")
for i, feature in enumerate(cat_col[1:]):
    encoded_cat_df = pd.concat([encoded_cat_df, pd.DataFrame(encoded_features[i+1].tolist()).add_prefix(f"{feature}_emb_")], axis=1)


: 

### Others

In [ ]:
# Define custom dataset class
class AnyDataset(Dataset):
    def __init__(self, cat_features, labels, tokenizer, max_length):
    #     self.num_features = num_features
        self.cat_features = cat_features
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.cat_features)

    def __getitem__(self, idx):
        text = ' '.join([str(feature) for feature in self.cat_features[idx]])
        label = self.labels[idx]

        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }



: 

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer

: 

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


: 

In [ ]:
def hyperparameter(train,MAX_LENGTH=128,BATCH_SIZE=32,num_epochs=5,label=None):
    """Initializing the Hyperparameters and Running the model

      parameters = dataset,
                    MAX_LENGTH,
                    BATCH_SIZE,
                    num_epochs,
                    label
                    """

    X_train, X_val, y_train, y_val = train_test_split(train,target,test_size=0.25,random_state=24)

    # Create custom datasets and data loaders
    train_dataset = AnyDataset(X_train, y_train, tokenizer, MAX_LENGTH)
    val_dataset = AnyDataset(X_val, y_val, tokenizer, MAX_LENGTH)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Fine-tune BERT model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train()

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    # print(next(iter(train_dataset)))

    torch.manual_seed(42)
    for epoch in range(num_epochs):
      for batch in train_loader:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['label'].to(device)

          outputs = model(input_ids, attention_mask=attention_mask)
          logits = outputs.last_hidden_state[:, 0, :]  # Get the CLS token representation
          # predictions = torch.argmax(logits, dim=1)
          prediction = outputs.Softmax(logits,dim=27)

          loss = torch.nn.L1Loss()(logits, labels)

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      print(f"Loss: {loss}")

      # Evaluate model on validation set
      model.eval()
      val_predictions = []
      val_labels = []

      for batch in val_loader:
          with torch.no_grad():
              input_ids = batch['input_ids'].to(device)
              attention_mask = batch['attention_mask'].to(device)
              labels = batch['label'].to(device)

              outputs = model(input_ids, attention_mask=attention_mask)
              logits = outputs.last_hidden_state[:, 0, :]
              # predictions = torch.argmax(logits, dim=1)
              predictions = outputs.Softmax(logits,dim=27)

              val_predictions.extend(predictions.cpu().numpy())
              val_labels.extend(labels.cpu().numpy())

      # Calculate accuracy
      # accuracy = log(val_labels, val_predictions)/
      # print(f'Validation Accuracy: {accuracy}')

    return {"model_name": model.__class__.__name__,
            "model_loss": loss.item(),
            # "model_acc": accuracy,
            'token_length':MAX_LENGTH,
            'batch_size':BATCH_SIZE}

: 

In [ ]:
hyperparameter(train_df,len(cat_col),label="SalePrice")

: 

: 